In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os

BASE1_path = '/kaggle/input/data-chap1-sql-intermediate/'

BASE2_path = '/kaggle/input/data-chap2-sql-intermediate/'

%sql sqlite://

'Connected: @None'

In [2]:
matches_2013_2014 = pd.read_table(os.path.join(BASE1_path, 'matches_2013_2014.txt'), sep = ';')
matches_2013_2014.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,1237,1,2013/2014,1,2014-03-29,8475,9989,2,0
1,1238,1,2013/2014,1,2014-03-29,9991,8573,0,1
2,1239,1,2013/2014,2,2014-04-05,9989,9991,1,0
3,1240,1,2013/2014,2,2014-04-05,8573,8475,0,0
4,1241,1,2013/2014,3,2014-04-12,9991,8475,2,1


In [3]:
team = pd.read_table(os.path.join(BASE2_path, 'team.txt'), sep = ';')
team.head()

,id,team_api_id,team_long_name,team_short_name
0,1,9987,KRC Genk,GEN
1,2,9993,Beerschot AC,BAC
2,3,10000,SV Zulte-Waregem,ZUL
3,4,9994,Sporting Lokeren,LOK
4,5,9984,KSV Cercle Brugge,CEB


In [4]:
match = pd.read_table(os.path.join(BASE1_path, 'match.txt'), sep = ';')
match.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,757,1,2011/2012,1,2011-07-29,1773,8635,2,1
1,758,1,2011/2012,1,2011-07-30,9998,9985,1,1
2,759,1,2011/2012,1,2011-07-30,9987,9993,3,1
3,760,1,2011/2012,1,2011-07-30,9991,9984,0,1
4,761,1,2011/2012,1,2011-07-30,9994,10000,0,0


In [5]:
country = pd.read_table(os.path.join(BASE1_path, 'EU_country.txt'), sep = ';')
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [6]:
league = pd.read_table(os.path.join(BASE2_path, 'league.txt'), sep = ';')
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


## create table

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////football_EU_league', echo=False)

match.to_sql('match', con = engine)
matches_2013_2014.to_sql('matches_2013_2014', con = engine)
country.to_sql('country', con = engine)
team.to_sql('team', con = engine)
league.to_sql('league', con = engine)

# PRACTICEs
## 1. Correlated subqueries

### 1.1. Basic Correlated Subqueries
Correlated subqueries are subqueries that reference one or more columns in the main query. Correlated subqueries depend on information in the main query to run, and thus, cannot be executed on their own.

Correlated subqueries are evaluated in **SQL** once per row of data retrieved -- a process that takes a lot more computing power and time than a simple subquery.

In this exercise, you will practice using correlated subqueries to examine matches with scores that are extreme outliers for each country -- above 3 times the average score!

#### Instructions

Select the `country_id, date, home_goal`, and `away_goal` columns in the main query.

Complete the **AVG** value in the subquery.

Complete the subquery column references, so that `country_id` is matched in the main and subquery.

In [8]:
pd.read_sql(
    """ 
        SELECT main.country_id, main.date, main.home_goal, main.away_goal
        FROM match AS main
        WHERE (home_goal + away_goal) > (SELECT AVG((sub.home_goal + sub.away_goal) * 3)
                                            FROM match AS sub
                                            WHERE main.country_id = sub.country_id);    
    """, con = engine)

,country_id,date,home_goal,away_goal
0,1,2011-10-29,4,5
1,1729,2011-08-28,8,2
2,1729,2012-12-29,7,3
3,1729,2013-05-19,5,5
4,1729,2013-12-14,6,3
5,1729,2014-03-22,3,6
6,1729,2014-08-30,3,6
7,4769,2011-10-15,5,3
8,4769,2011-12-21,4,4
9,4769,2012-02-12,4,5


`Correlated subqueries` take **longer** to produce results, ***but*** they often **prevent** you from having to create `multiple subqueries`.

### 1.2 Correlated subquery with multiple conditions
Correlated subqueries are useful for matching data across multiple columns. In the previous exercise, you generated a list of matches with extremely high scores for each `country`. In this exercise, you're going to add an additional column for matching to answer the question -- what was the highest scoring match for each `country`, in each season?

#### Instructions
Select `the country_id, date, home_goal`, and `away_goal` columns in the main query.

Complete the subquery: Select the matches with the highest number of total goals.

Match the subquery to the main query using `country_id` and `season`.

Fill in the correct logical operator so that total goals equals the max goals recorded in the subquery.

In [9]:
pd.read_sql(
    """ 
        SELECT main.country_id, main.date, main.home_goal, main.away_goal
        FROM match AS main
        WHERE (home_goal + away_goal) = (SELECT MAX(sub.home_goal + sub.away_goal)
                                         FROM match AS sub
                                         WHERE main.season = sub.season
                                               AND main.country_id = sub.country_id);    
    """, con = engine)

,country_id,date,home_goal,away_goal
0,1,2011-10-29,4,5
1,1,2012-11-17,2,6
2,1,2012-12-09,1,7
3,1,2013-01-19,2,6
4,1,2012-08-19,2,6
...,...,...,...,...
73,24558,2012-09-30,6,2
74,24558,2014-02-16,5,3
75,24558,2015-04-30,6,2
76,24558,2015-05-03,2,6


`Subqueries` can be very useful for selecting data broken into multiple categories -- though you may have to wait for the data to load!

## 2. Nested Subqueries

### 2.1. Nested simple subqueries
`Nested subqueries` can be either simple or correlated.

Just like an `unnested subquery`, a `nested subquery's components` can be executed independently of the outer query, while a correlated subquery requires both the outer and inner subquery to run and produce results.

In this exercise, you will practice creating a nested subquery to examine the `highest total` number of goals in each `season`, overall, and during `July` across all `seasons`.

#### Instructions
Complete the main query to select the `season` and the `max total goals` in a match for each `season`. Name this `max_goals`.

Complete the first simple subquery to select the `max total goals` in a match across all seasons. Name this `overall_max_goals`.

Complete the nested subquery to select the maximum total goals in a match played in July across all seasons.

Select the `maximum total goals` in the outer subquery. Name this entire subquery `july_max_goals`.

In [10]:
pd.read_sql(
    """ 
        SELECT season,
               MAX(home_goal + away_goal) AS max_goals,
               (SELECT MAX(home_goal + away_goal) 
                   FROM match) AS overall_max_goals,
               (SELECT MAX(home_goal + away_goal) 
                   FROM match
                   WHERE id IN ( SELECT id 
                                   FROM match 
                                   WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max_goals
        FROM match
        GROUP BY season;    
    """, con = engine)

,season,max_goals,overall_max_goals,july_max_goals
0,2013/2014,10,11,7
1,2012/2013,11,11,7
2,2014/2015,10,11,7
3,2011/2012,10,11,7


### 2.2. Nest a subquery in FROM

What's the average number of matches per season where a **team scored 6 or more goals**? How does this differ by country?

Let's use a nested, correlated subquery to perform this operation. In the real world, you will probably find that nesting multiple subqueries is a task you don't have to perform often. In some cases, however, you may find yourself struggling to properly group by the column you want, or to calculate information requiring multiple mathematical transformations (i.e., an **AVG** of a **COUNT**).

Nesting subqueries and performing your transformations one step at a time, adding it to a subquery, and then performing the next set of transformations is often the easiest way to yield accurate information about your data. Let's get to it!

#### Instructions.

**Step 1.** Generate a list of matches where at least one team scored 5 or more goals.

In [11]:
pd.read_sql(
    """ 
        SELECT country_id, season, id
        FROM match
        WHERE home_goal > 6 OR away_goal > 6;    
    """, con = engine)

,country_id,season,id
0,1,2012/2013,1080
1,1,2014/2015,1363
2,1729,2011/2012,3029
3,1729,2011/2012,3093
4,1729,2012/2013,3339
5,1729,2012/2013,3369
6,1729,2013/2014,3644
7,1729,2014/2015,4376
8,4769,2014/2015,7073
9,7809,2011/2012,8880


**Step 2.** Turn the query from the previous step into a `subquery` in the **FROM** statement.

**COUNT** the match ids generated in the previous step, and group the `query` by `country_id` and `season`.

In [12]:
pd.read_sql(
    """ 
        SELECT country_id, season, COUNT(id) AS matches
        FROM ( SELECT country_id, season, id
                FROM match
                WHERE home_goal >= 6 OR away_goal >= 6
            ) AS subquery
        GROUP BY season, country_id;    
    """, con = engine)

,country_id,season,matches
0,15722,2011/2012,1
1,1729,2011/2012,6
2,24558,2011/2012,3
3,19694,2012/2013,1
4,4769,2011/2012,2
5,17642,2014/2015,2
6,10257,2013/2014,1
7,7809,2012/2013,5
8,17642,2012/2013,1
9,19694,2014/2015,2


**Step 3.** Finally, declare the same query from **step 2** as a `subquery` in **FROM** with the alias `outer_s`.

**Left join** it to the `country` table using the outer query's `country_id column`.

Calculate an **AVG** of high scoring matches per `country` in the main `query`.

In [13]:
pd.read_sql(
    """ 
        SELECT c.name AS country, 
               AVG(c.id) AS avg_seasonal_high_scores
        FROM country AS c
        LEFT JOIN ( SELECT country_id, season,
                           COUNT(id) AS matches
                    FROM ( SELECT country_id, season, id
                              FROM match
                              WHERE home_goal >= 5 OR away_goal >= 5
                              ) AS inner_s  
                    GROUP BY country_id, season) AS outer_s
            ON c.id = outer_s.country_id
        GROUP BY country;         
    """, con = engine)

,country,avg_seasonal_high_scores
0,Portugal,17642.0
1,France,4769.0
2,Scotland,19694.0
3,Netherlands,13274.0
4,Spain,21518.0
5,Belgium,1.0
6,Italy,10257.0
7,Germany,7809.0
8,England,1729.0
9,Switzerland,24558.0


This required quite a few steps, but you've got the hang of steps necessary to nest subqueries in **FROM** that you can apply to other parts of your query as well!

## 3. Common Table Expressions (CTEs)
### 3.1. Clean up with CTEs
In chapter 2, you generated a list of countries and the number of matches in each country with more than 10 total goals. The query in that exercise utilized a subquery in the **FROM** statement in order to filter the matches before counting them in the main query. Below is the query you created:

                    SELECT
                      c.name AS country,
                      COUNT(sub.id) AS matches
                    FROM country AS c
                    INNER JOIN (
                      SELECT country_id, id 
                      FROM match
                      WHERE (home_goal + away_goal) >= 10) AS sub
                    ON c.id = sub.country_id
                    GROUP BY country;
You can list one (or more) subqueries as **common table expressions (`CTEs`)** by declaring them ahead of your main query, which is an excellent tool for organizing information and placing it in a logical order.

In this exercise, let's rewrite a similar query using a `CTE`.

#### Instructions
Complete the syntax to declare your `CTE`.

Select the `country_id` and `match id` from the `match` table in your `CTE`.

**Left join** the `CTE` to the league table using `country_id`. 

In [14]:
pd.read_sql(
    """ 
        WITH match_list AS ( SELECT country_id, id
                             FROM match
                             WHERE (home_goal + away_goal) >= 10
                             )
       SELECT l.name AS league,
              COUNT(match_list.id) AS matches
       FROM league AS l
       LEFT JOIN match_list 
           ON l.id = match_list.country_id
       GROUP BY l.name;    
    """, con = engine)

,league,matches
0,Switzerland Super League,0
1,Poland Ekstraklasa,0
2,Netherlands Eredivisie,1
3,Scotland Premier League,0
4,France Ligue 1,0
5,Spain LIGA BBVA,4
6,Germany 1. Bundesliga,1
7,Italy Serie A,0
8,Portugal Liga ZON Sagres,0
9,England Premier League,3


Isn't this much easier to read? We just rearranged a query to produce similar results to a subquery in **FROM**!

### 3.2. Organizing with CTEs
Previously, you modified a query based on a statement you completed in chapter 2 using `common table expressions`.

This time, let's expand on the exercise by looking at details about matches with very high scores using `CTEs`. Just like a subquery in **FROM**, you can join tables inside a `CTE`.

#### Instructions
Declare your `CTE`, where you create a list of all matches with the `league` name.

Select the `league`, `date`, `home`, and `away goals` from the `CTE`.

Filter the main query for matches with **10 or more `goals`**.

In [15]:
pd.read_sql(
    """ 
        WITH match_list AS ( SELECT l.name AS league, m.date, m.home_goal, m.away_goal,
                               (m.home_goal + m.away_goal) AS total_goals
                             FROM match AS m
                             LEFT JOIN league as l 
                                ON m.country_id = l.id
                             )
       -- Select the league, date, home, and away goals from the CTE
        SELECT league, date, home_goal, away_goal
        FROM match_list
        
        -- Filter by total goals
        WHERE total_goals >= 10;    
    """, con = engine)

,league,date,home_goal,away_goal
0,England Premier League,2011-08-28,8,2
1,England Premier League,2012-12-29,7,3
2,England Premier League,2013-05-19,5,5
3,Germany 1. Bundesliga,2013-03-30,9,2
4,Netherlands Eredivisie,2011-11-06,6,4
5,Spain LIGA BBVA,2013-10-30,7,3
6,Spain LIGA BBVA,2015-04-05,9,1
7,Spain LIGA BBVA,2015-05-23,7,3
8,Spain LIGA BBVA,2014-09-20,2,8


CTEs are much easier to read than multiple subqueries!

### 3.3. CTEs with nested subqueries
If you find yourself listing multiple subqueries in the **FROM** clause with nested statement, your query will likely become long, complex, and difficult to read.

Since many queries are written with the intention of being saved and re-run in the future, proper organization is key to a seamless workflow. 

Arranging subqueries as `CTEs` will save you time, space, and confusion in the long run!

#### Instructions

Declare a `CTE` that calculates the total goals from matches in `August` of the `2013/2014 season`.

**Left join** the `CTE` onto the `league` table using `country_id` from the `match_list CTE`.

Filter the list on the inner subquery to only select matches in `August` of the `2013/2014 season`.

In [16]:
pd.read_sql(
    """ 
        WITH match_list AS ( SELECT country_id,
                                   (home_goal + away_goal) AS goals
                            FROM match
                            WHERE id IN ( SELECT id
                                          FROM match
                                          WHERE season = '2013/2014' AND 
                                                EXTRACT(MONTH FROM date) = 8))
        SELECT name, AVG(match_list.goals)
        FROM league AS l
        LEFT JOIN match_list 
            ON l.id = match_list.country_id
        GROUP BY l.name;    
    """, con = engine)

,name,avg
0,Switzerland Super League,1.937500
1,Poland Ekstraklasa,2.310345
2,Netherlands Eredivisie,3.414634
3,Scotland Premier League,2.137931
4,France Ligue 1,2.027027
5,Spain LIGA BBVA,2.920000
6,Germany 1. Bundesliga,3.235294
7,Italy Serie A,2.750000
8,Portugal Liga ZON Sagres,3.000000
9,England Premier League,2.000000


## 4.Deciding on techniques to use

### 4.1. Problem 1. Get team names with a subquery
Let's solve a problem we've encountered a few times in this course so far -- How do you get both the home and away team names into one final query result?

Out of the 4 techniques we just discussed, this can be performed using subqueries, correlated subqueries, and CTEs. Let's practice creating similar result sets using each of these 3 methods over the next 3 exercises, starting with subqueries in **FROM**.

#### Instructions 
**Step 1.** Create a query that left joins `team` to `match` in order to get the identity of the home team. This becomes the subquery in the next step.

In [17]:
pd.read_sql(
    """ 
        SELECT m.id,t.team_long_name AS hometeam
        FROM match AS m
        LEFT JOIN team as t
        ON m.hometeam_id = team_api_id
        LIMIT 100
    """, con = engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren
...,...,...
95,852,KSV Cercle Brugge
96,853,RSC Anderlecht
97,854,Standard de Liège
98,855,Sint-Truidense VV


**Step 2.** Add a second subquery to the **FROM** statement to get the away team name, changing only the `hometeam_id`. 

**Left join** both `subqueries` to the `match` table on the `id` column.

In [18]:
pd.read_sql(
    """ 
        SELECT m.date, hometeam, awayteam, m.home_goal, m.away_goal
        FROM match AS m
        LEFT JOIN ( SELECT match.id, team.team_long_name AS hometeam
                    FROM match
                    LEFT JOIN team
                        ON match.hometeam_id = team.team_api_id) AS home
            ON home.id = m.id

        -- Join the away subquery to the match table
        LEFT JOIN ( SELECT match.id, team.team_long_name AS awayteam
                    FROM match
                    LEFT JOIN team
                      -- Get the away team ID in the subquery
                      ON match.awayteam_id = team.team_api_id) AS away
            ON away.id = m.id
        LIMIT 100
    """, con = engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30,RAEC Mons,Standard de Liège,1,1
2,2011-07-30,KRC Genk,Beerschot AC,3,1
3,2011-07-30,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
95,2011-08-06,KSV Cercle Brugge,RAEC Mons,1,1
96,2012-01-15,RSC Anderlecht,Club Brugge KV,3,0
97,2012-01-14,Standard de Liège,Beerschot AC,6,1
98,2012-01-14,Sint-Truidense VV,KV Kortrijk,0,0


This is one clear way to generate a list of team names who played in a `match`. This is much easier to read than `team ID` numbers!

### 4.2 Get team names with correlated subqueries
Let's solve the same problem using correlated subqueries -- How do you get both the home and away team names into one final query result?

This can easily be performed using correlated subqueries. But how might that impact the performance of your query? Complete the following steps and let's find out!

Please note that your query will run more slowly than the previous exercise!

#### Instructions
**Step 1.** Using a correlated subquery in the **SELECT** statement, match the `team_api_id` column from team to the `hometeam_id` from match.

In [19]:
pd.read_sql(
    """ 
        SELECT m.date,
                (SELECT team_long_name
                 FROM team AS t
                 -- Connect the team to the match table
                 WHERE team_api_id = m.hometeam_id
                ) AS hometeam
        FROM match AS m
        LIMIT 100
    """, con = engine)

,date,hometeam
0,2011-07-29,Oud-Heverlee Leuven
1,2011-07-30,RAEC Mons
2,2011-07-30,KRC Genk
3,2011-07-30,KAA Gent
4,2011-07-30,Sporting Lokeren
...,...,...
95,2011-08-06,KSV Cercle Brugge
96,2012-01-15,RSC Anderlecht
97,2012-01-14,Standard de Liège
98,2012-01-14,Sint-Truidense VV


**Step 2.** Create a second correlated subquery in **SELECT**, yielding the `away team`'s name.

Select the `home` and `away goal` columns from `match` in the main `query`.

In [20]:
pd.read_sql(
    """ 
        SELECT m.date,
               ( SELECT team_long_name
                 FROM team AS t
                 WHERE t.team_api_id = m.hometeam_id
                ) AS hometeam,
             
             -- Connect the team to the match table
              ( SELECT team_long_name
                FROM team AS t
                WHERE team_api_id = m.awayteam_id
              ) AS awayteam,
            
            -- Select home and away goals
             m.home_goal, m.away_goal
             
        FROM match AS m
        LIMIT 100
    """, con = engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30,RAEC Mons,Standard de Liège,1,1
2,2011-07-30,KRC Genk,Beerschot AC,3,1
3,2011-07-30,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
95,2011-08-06,KSV Cercle Brugge,RAEC Mons,1,1
96,2012-01-15,RSC Anderlecht,Club Brugge KV,3,0
97,2012-01-14,Standard de Liège,Beerschot AC,6,1
98,2012-01-14,Sint-Truidense VV,KV Kortrijk,0,0


The syntax here is much simpler, but if you're working with a large data set your query may take a while to run!

### 4.Get team names with CTEs
You've now explored two methods for answering the question, How do you get both the home and away team names into one final query result?

Let's explore the final method - common table expressions. Common table expressions are similar to the subquery method for generating results, mainly differing in syntax and the order in which information is processed.

#### Instructions
**Step 1.** Select `id` from `match` and `team_long_name` from `team`. **Join** these two tables together on `hometeam_id` in `match` and `team_api_id` in `team`.

In [21]:
pd.read_sql(
    """ 
        SELECT m.id, t.team_long_name AS hometeam
        FROM match AS m
        LEFT JOIN team AS t 
            ON t.team_api_id = m.hometeam_id
        LIMIT 100;  
    """, con = engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren
...,...,...
95,852,KSV Cercle Brugge
96,853,RSC Anderlecht
97,854,Standard de Liège
98,855,Sint-Truidense VV


**Step 2.** Declare the query from the previous step as a common table expression. **SELECT everything** from the `CTE` using **WITH** and **AS**  into the `main query`. 

Your results will not change at this step!

In [22]:
pd.read_sql(
    """ 
        WITH home AS (SELECT m.id, t.team_long_name AS hometeam
                        FROM match AS m
                        LEFT JOIN team AS t 
                            ON m.hometeam_id = t.team_api_id)
        -- Select everything from home
        SELECT *FROM home
        LIMIT 100
    """, con = engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren
...,...,...
95,852,KSV Cercle Brugge
96,853,RSC Anderlecht
97,854,Standard de Liège
98,855,Sint-Truidense VV


**Step 3.** Let's declare the **second CTE**, `away`. **Join** it to the `first CTE` on the `id` column.

The `date`, `home_goal`, and `away_goal` columns have been added to the `CTEs`. **SELECT** them into the `main query`.

In [23]:
pd.read_sql(
    """ 
        WITH home AS( SELECT m.id, m.date, 
                             t.team_long_name AS hometeam, m.home_goal
                        FROM match AS m
                        LEFT JOIN team AS t 
                            ON m.hometeam_id = t.team_api_id),
        -- Declare and set up the away CTE
        away AS ( SELECT m.id, m.date, 
                         t.team_long_name AS awayteam, m.away_goal
                  FROM match AS m
                  LEFT JOIN team AS t 
                      ON m.awayteam_id = t.team_api_id)
        -- Select date, home_goal, and away_goal
        SELECT home.date, home.hometeam, away.awayteam, home.home_goal, away.away_goal
        FROM home
        INNER JOIN away
            ON home.id = away.id
        LIMIT 100;    
    """, con = engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30,RAEC Mons,Standard de Liège,1,1
2,2011-07-30,KRC Genk,Beerschot AC,3,1
3,2011-07-30,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
95,2011-08-06,KSV Cercle Brugge,RAEC Mons,1,1
96,2012-01-15,RSC Anderlecht,Club Brugge KV,3,0
97,2012-01-14,Standard de Liège,Beerschot AC,6,1
98,2012-01-14,Sint-Truidense VV,KV Kortrijk,0,0


You now know three separate ways to manipulate and transform data to produce a complex query result! This is a great set of skills to have when working with complex relational databases!

Based on what you learned, which of the following statements is **NOT** true regarding differences in the use and performance of multiple/nested subqueries, correlated subqueries, and `common table expressions`?

> A. Correlated subqueries can allow you to circumvent multiple, complex joins.
>
> B. Common table expressions are declared first, improving query run time.
> 
> **C. Correlated subqueries can reduce the length of your query, improving query run time.**
>
> D. Multiple or nested subqueries are processed first, before your main query.